# Chest X-Ray Dataset Exploratory Data Analysis (EDA)

**Objective**: Comprehensive analysis of ChestX-ray14 dataset to understand class distribution, imbalance, and inform preprocessing/training strategy.

**Dataset**: NIH ChestX-ray14 (112,120 frontal-view X-rays, 14 disease classes, multi-label)

**Date**: January 21, 2026

---

## 1. Setup & Imports

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set project root
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Import project config
from config import Config, DISEASE_LABELS, NUM_CLASSES

print(f"Project Root: {project_root}")
print(f"Number of Disease Classes: {NUM_CLASSES}")
print(f"Disease Classes: {DISEASE_LABELS}")

## 2. Load Dataset Metadata

In [ ]:
# Load metadata CSV
metadata_path = Config.METADATA_CSV
print(f"Loading metadata from: {metadata_path}")

df = pd.read_csv(metadata_path)

print(f"\n✓ Dataset loaded successfully!")
print(f"  Total records: {len(df):,}")
print(f"  Columns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
df.head()

## 3. Dataset Basic Statistics

In [ ]:
print("="*70)
print("DATASET STATISTICS")
print("="*70)

# Column names in metadata
print(f"\nDataframe Info:")
print(df.info())

print(f"\nBasic Statistics:")
print(f"  Total Images: {len(df):,}")
print(f"  Unique Image IDs: {df.iloc[:, 0].nunique():,}")
print(f"  Patients: {df.iloc[:, 4].nunique():,}" if df.shape[1] > 4 else "  N/A")

print(f"\nDataset Overview:")
print(df.describe())

## 4. Disease Label Analysis

In [ ]:
# Parse disease labels (column 1 contains pipe-separated labels)
label_column = df.iloc[:, 1]  # Typically column 1 (index 1) has labels

# Count each disease occurrence
disease_counts = {}
total_labels = 0
images_with_no_finding = 0

for labels in label_column:
    if pd.isna(labels):
        images_with_no_finding += 1
        continue
    
    label_str = str(labels).strip()
    
    if label_str == 'No Finding':
        images_with_no_finding += 1
        total_labels += 1
    else:
        # Split by pipe and count
        diseases = [d.strip() for d in label_str.split('|')]
        for disease in diseases:
            disease_counts[disease] = disease_counts.get(disease, 0) + 1
            total_labels += 1

print("="*70)
print("DISEASE LABEL DISTRIBUTION")
print("="*70)

# Sort by count
sorted_diseases = sorted(disease_counts.items(), key=lambda x: x[1], reverse=True)

print(f"\nTotal Disease Occurrences: {total_labels:,}")
print(f"Images with 'No Finding': {images_with_no_finding:,} ({images_with_no_finding/len(df)*100:.1f}%)")
print(f"\nDisease Counts:")
print("-"*70)
print(f"{'Rank':<6} {'Disease':<30} {'Count':<12} {'Percentage':<12}")
print("-"*70)

for rank, (disease, count) in enumerate(sorted_diseases, 1):
    percentage = (count / len(df)) * 100
    print(f"{rank:<6} {disease:<30} {count:<12,} {percentage:<12.2f}%")

print("-"*70)

## 5. Class Imbalance Metrics

In [ ]:
# Calculate imbalance metrics
print("\n" + "="*70)
print("CLASS IMBALANCE ANALYSIS")
print("="*70)

# Get counts
counts = np.array([count for _, count in sorted_diseases])

# Metrics
min_count = counts.min()
max_count = counts.max()
imbalance_ratio = max_count / min_count

print(f"\nImbalance Metrics:")
print(f"  Max count: {max_count:,}")
print(f"  Min count: {min_count:,}")
print(f"  Imbalance ratio (max/min): {imbalance_ratio:.2f}:1")

# Gini coefficient (measure of inequality)
def gini_coefficient(counts):
    n = len(counts)
    sorted_counts = np.sort(counts)
    cumsum = np.cumsum(sorted_counts)
    gini = (2 * np.sum(np.arange(1, n+1) * sorted_counts)) / (n * np.sum(sorted_counts)) - (n + 1) / n
    return gini

gini = gini_coefficient(counts)
print(f"  Gini coefficient: {gini:.4f} (0=perfect balance, 1=extreme imbalance)")

# Effective number of samples (recommended for imbalanced data)
def effective_num_samples(counts, beta=0.9999):
    return (1 - beta) / (1 - (beta ** counts))

effective_counts = effective_num_samples(counts)
print(f"  Effective number of samples (β=0.9999):")
for (disease, _), eff in zip(sorted_diseases, effective_counts):
    print(f"    {disease}: {eff:.0f}")

# Class weights (inverse frequency)
class_weights = {disease: len(df) / (count * len(sorted_diseases)) 
                 for disease, count in sorted_diseases}

print(f"\n  Recommended class weights (inverse frequency):")
for disease, weight in sorted(class_weights.items(), key=lambda x: x[1], reverse=True):
    print(f"    {disease}: {weight:.4f}")

## 6. Multi-Label Statistics

In [ ]:
# Count diseases per image
diseases_per_image = []

for labels in label_column:
    if pd.isna(labels):
        diseases_per_image.append(0)
    else:
        label_str = str(labels).strip()
        if label_str == 'No Finding':
            diseases_per_image.append(0)
        else:
            disease_count = len([d.strip() for d in label_str.split('|') if d.strip()])
            diseases_per_image.append(disease_count)

df['num_diseases'] = diseases_per_image

print("\n" + "="*70)
print("MULTI-LABEL STATISTICS")
print("="*70)

print(f"\nDiseases per image distribution:")
dist = df['num_diseases'].value_counts().sort_index()
for num_diseases, count in dist.items():
    percentage = (count / len(df)) * 100
    print(f"  {num_diseases} disease(s): {count:>6,} images ({percentage:>5.1f}%)")

print(f"\nSummary:")
print(f"  Mean diseases per image: {df['num_diseases'].mean():.2f}")
print(f"  Median diseases per image: {df['num_diseases'].median():.0f}")
print(f"  Max diseases per image: {df['num_diseases'].max():.0f}")
print(f"  Min diseases per image: {df['num_diseases'].min():.0f}")
print(f"  Std dev: {df['num_diseases'].std():.2f}")

print(f"\nCritical finding:")
no_disease = len(df[df['num_diseases'] == 0])
multi_disease = len(df[df['num_diseases'] > 1])
print(f"  Normal (0 diseases): {no_disease:,} ({no_disease/len(df)*100:.1f}%)")
print(f"  Multi-pathology (>1 disease): {multi_disease:,} ({multi_disease/len(df)*100:.1f}%)")

## 7. Disease Co-Occurrence Analysis

In [ ]:
# Build co-occurrence matrix
from itertools import combinations

co_occurrence = {}

for labels in label_column:
    if pd.isna(labels):
        continue
    
    label_str = str(labels).strip()
    if label_str == 'No Finding':
        continue
    
    diseases = [d.strip() for d in label_str.split('|')]
    
    # Count pairs
    for disease_pair in combinations(sorted(diseases), 2):
        key = tuple(sorted(disease_pair))
        co_occurrence[key] = co_occurrence.get(key, 0) + 1

print("\n" + "="*70)
print("DISEASE CO-OCCURRENCE (Top 15 pairs)")
print("="*70)

sorted_pairs = sorted(co_occurrence.items(), key=lambda x: x[1], reverse=True)[:15]

print(f"\n{'Rank':<6} {'Disease 1':<20} {'Disease 2':<20} {'Co-occurrence':<12}")
print("-"*70)

for rank, ((disease1, disease2), count) in enumerate(sorted_pairs, 1):
    print(f"{rank:<6} {disease1:<20} {disease2:<20} {count:<12,}")

print("-"*70)
print(f"\nTotal unique disease pairs: {len(co_occurrence)}")

## 8. Image File Validation

In [ ]:
print("\n" + "="*70)
print("IMAGE FILE VALIDATION")
print("="*70)

image_dir = Config.RAW_DATA_DIR / 'images'

if not image_dir.exists():
    print(f"\n⚠️  Image directory not found: {image_dir}")
    print(f"Expected path: {Config.RAW_DATA_DIR}")
    print(f"\nPlease ensure images are extracted to: {image_dir}")
else:
    # Count actual image files
    image_files = list(image_dir.glob("*.png"))
    image_count = len(image_files)
    
    print(f"\n✓ Image directory found: {image_dir}")
    print(f"  Total image files: {image_count:,}")
    print(f"  Expected from metadata: {len(df):,}")
    
    if image_count == len(df):
        print(f"  ✓ Counts match!")
    else:
        print(f"  ⚠️  Count mismatch! ({len(df) - image_count:,} missing)")
    
    # Sample image info
    if image_files:
        print(f"\n  Sample image info:")
        for img_path in image_files[:3]:
            try:
                img = Image.open(img_path)
                print(f"    {img_path.name}: {img.size} pixels, mode={img.mode}")
            except Exception as e:
                print(f"    {img_path.name}: ERROR - {e}")

## 9. Visualizations

In [ ]:
# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 12)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Disease Distribution (Bar chart)
diseases = [d for d, _ in sorted_diseases]
counts_list = [c for _, c in sorted_diseases]
colors = plt.cm.viridis(np.linspace(0, 1, len(diseases)))

axes[0, 0].barh(diseases, counts_list, color=colors)
axes[0, 0].set_xlabel('Number of Cases', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Disease Distribution Across Dataset', fontsize=14, fontweight='bold')
axes[0, 0].invert_yaxis()

for i, v in enumerate(counts_list):
    axes[0, 0].text(v, i, f' {v:,}', va='center', fontsize=9)

# Plot 2: Disease Percentage
percentages = [(c/len(df)*100) for c in counts_list]
axes[0, 1].bar(range(len(diseases)), percentages, color=colors)
axes[0, 1].set_xticks(range(len(diseases)))
axes[0, 1].set_xticklabels(diseases, rotation=45, ha='right')
axes[0, 1].set_ylabel('Percentage (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Prevalence Rate (% of Dataset)', fontsize=14, fontweight='bold')

# Plot 3: Diseases per Image
disease_dist = df['num_diseases'].value_counts().sort_index()
axes[1, 0].bar(disease_dist.index, disease_dist.values, color='steelblue', edgecolor='black')
axes[1, 0].set_xlabel('Number of Diseases per Image', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Multi-Label Distribution', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Plot 4: Class Weight Distribution (Log scale)
weights = [class_weights[d] for d in diseases]
axes[1, 1].barh(diseases, weights, color=colors)
axes[1, 1].set_xlabel('Class Weight (Inverse Frequency)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Recommended Class Weights for Training', fontsize=14, fontweight='bold')
axes[1, 1].set_xscale('log')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.savefig('../../results/01_eda_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualizations saved to: results/01_eda_overview.png")

## 10. Sample Visualization

In [ ]:
# Display sample images with their labels
image_dir = Config.RAW_DATA_DIR / 'images'

if image_dir.exists() and len(list(image_dir.glob("*.png"))) > 0:
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.flatten()
    
    # Get sample images with different disease counts
    samples = [
        df[df['num_diseases'] == 0].sample(1, random_state=42).iloc[0],  # Normal
        df[df['num_diseases'] == 1].sample(1, random_state=42).iloc[0],  # Single disease
        df[df['num_diseases'] == 2].sample(1, random_state=42).iloc[0],  # 2 diseases
        df[df['num_diseases'] == 0].sample(1, random_state=43).iloc[0],
        df[df['num_diseases'] == 1].sample(1, random_state=43).iloc[0],
        df[df['num_diseases'] == 2].sample(1, random_state=43).iloc[0],
        df[df['num_diseases'] == 0].sample(1, random_state=44).iloc[0],
        df[df['num_diseases'] == 1].sample(1, random_state=44).iloc[0],
        df[df['num_diseases'] == 2].sample(1, random_state=44).iloc[0],
    ]
    
    for idx, (ax, sample) in enumerate(zip(axes, samples)):
        image_id = sample.iloc[0]
        image_path = image_dir / image_id
        
        labels = sample.iloc[1]
        if pd.isna(labels):
            labels = 'No Finding'
        
        try:
            img = Image.open(image_path)
            ax.imshow(img, cmap='gray')
            ax.set_title(f"ID: {image_id}\nLabels: {labels}", fontsize=9, wrap=True)
            ax.axis('off')
        except Exception as e:
            ax.text(0.5, 0.5, f'Error loading:\n{image_id}', ha='center')
            ax.axis('off')
    
    plt.tight_layout()
    plt.savefig('../../results/02_sample_images.png', dpi=200, bbox_inches='tight')
    plt.show()
    
    print("✓ Sample images saved to: results/02_sample_images.png")
else:
    print("⚠️  Images not available for visualization")

## 11. Stratification Strategy

In [ ]:
from sklearn.model_selection import train_test_split

print("\n" + "="*70)
print("DATA STRATIFICATION STRATEGY")
print("="*70)

# Use disease count as stratification variable
stratify_by = df['num_diseases']

# Split: 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(
    df, 
    test_size=0.3, 
    stratify=stratify_by,
    random_state=42
,
,
,
,
,
,
,
,
,
,
\nDataset Split:")
print(f"  Training set: {len(train_df):,} images ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validation set: {len(val_df):,} images ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Test set: {len(test_df):,} images ({len(test_df)/len(df)*100:.1f}%)")

print(f"\nStratification Validation (Multi-label distribution):")
print("-"*70)

for num_dis in sorted(df['num_diseases'].unique()):
    train_count = len(train_df[train_df['num_diseases'] == num_dis])
    val_count = len(val_df[val_df['num_diseases'] == num_dis])
    test_count = len(test_df[test_df['num_diseases'] == num_dis])
    
    total = train_count + val_count + test_count
    train_pct = train_count/total*100 if total > 0 else 0
    val_pct = val_count/total*100 if total > 0 else 0
    test_pct = test_count/total*100 if total > 0 else 0
    
    print(f"{num_dis} disease(s): Train {train_pct:.1f}% | Val {val_pct:.1f}% | Test {test_pct:.1f}%")

print("-"*70)
print("\n✓ Stratification successful - distributions preserved across splits!")

## 12. Class Imbalance Handling Recommendations

In [ ]:
print("\n" + "="*70)
print("CLASS IMBALANCE MITIGATION STRATEGY")
print("="*70)

print("""
Based on EDA findings, here's the recommended approach:

1. LOSS FUNCTION
   ✓ Use BCEWithLogitsLoss (for multi-label binary cross-entropy)
   ✓ Apply class weights (inverse frequency) from above
   ✓ Consider Focal Loss: reduces impact of easy negatives
   
2. SAMPLING STRATEGY
   ✓ Use WeightedRandomSampler during training
   ✓ Ensures each batch has balanced disease representation
   ✓ Prevents model overfitting to majority class
   
3. AUGMENTATION
   ✓ Apply stronger augmentation to minority classes
   ✓ Horizontal flip: safe for chest X-rays
   ✓ Rotation ±15°, slight brightness/contrast changes
   ✓ Skip extreme augmentations (vertical flip, 90° rotation dangerous)
   
4. TRAINING CONFIGURATION
   ✓ Use stratified k-fold cross-validation
   ✓ Monitor per-class AUC-ROC (not just accuracy)
   ✓ Use macro-averaged F1 for evaluation
   ✓ Set decision threshold per class (don't use 0.5 for all)
   
5. VALIDATION METRICS
   ✓ Primary: Per-class AUC-ROC (target >0.80)
   ✓ Secondary: F1-score (macro and weighted)
   ✓ Supplementary: Precision, Recall per class
   ✓ Ignore: Accuracy (misleading with imbalanced data)
""")

print("="*70)

## 13. Export Processed Splits

In [ ]:
# Create splits directory
splits_dir = project_root / 'data' / 'splits'
splits_dir.mkdir(exist_ok=True)

# Save splits
train_path = splits_dir / 'train.csv'
val_path = splits_dir / 'val.csv'
test_path = splits_dir / 'test.csv'

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("\n" + "="*70)
print("DATA SPLITS EXPORTED")
print("="*70)

print(f"\n✓ Training split: {train_path}")
print(f"  Samples: {len(train_df):,}")

print(f"\n✓ Validation split: {val_path}")
print(f"  Samples: {len(val_df):,}")

print(f"\n✓ Test split: {test_path}")
print(f"  Samples: {len(test_df):,}")

print(f"\n✓ All splits saved to: {splits_dir}")

## 14. Summary & Key Findings

In [ ]:
summary = f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    EXPLORATORY DATA ANALYSIS SUMMARY                         ║
╚══════════════════════════════════════════════════════════════════════════════╝

📊 DATASET OVERVIEW
   • Total Images: {len(df):,}
   • Disease Classes: {len(sorted_diseases)} (excluding No Finding)
   • Multi-label: {len(df[df['num_diseases'] > 1]):,} images ({len(df[df['num_diseases'] > 1])/len(df)*100:.1f}%)
   • Normal findings: {images_with_no_finding:,} images ({images_with_no_finding/len(df)*100:.1f}%)

⚠️  CLASS IMBALANCE SEVERITY
   • Imbalance ratio: {imbalance_ratio:.1f}:1 (high)
   • Gini coefficient: {gini:.4f} (significant inequality)
   • Most prevalent: {sorted_diseases[0][0]} ({sorted_diseases[0][1]:,} cases)
   • Least prevalent: {sorted_diseases[-1][0]} ({sorted_diseases[-1][1]:,} cases)

📈 MULTI-LABEL DISTRIBUTION
   • Average diseases per image: {df['num_diseases'].mean():.2f}
   • Single disease prevalence: {len(df[df['num_diseases']==1])/len(df)*100:.1f}%
   • Multiple disease prevalence: {len(df[df['num_diseases']>1])/len(df)*100:.1f}%

🔧 MITIGATION STRATEGIES IMPLEMENTED
   ✓ Stratified train/val/test split (preserves distribution)
   ✓ Inverse frequency class weights calculated
   ✓ Per-class evaluation metrics identified
   ✓ Balanced sampling strategy recommended

💾 DATA READY FOR PREPROCESSING
   ✓ Metadata validated: {len(df):,} records
   ✓ Images verified: {len(list(image_dir.glob('*.png'))):,} files (if downloaded)
   ✓ Splits created: train/{len(train_df):,} | val/{len(val_df):,} | test/{len(test_df):,}
   ✓ Export location: {splits_dir}

⚡ NEXT STEPS
   1. Create data loader using stratified splits
   2. Implement class-weighted sampling in DataLoader
   3. Configure loss function with class weights
   4. Setup per-class evaluation metrics
   5. Begin model training (preferably with focal loss)

═══════════════════════════════════════════════════════════════════════════════
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
═══════════════════════════════════════════════════════════════════════════════
"""

print(summary)

# Save summary to file
with open(project_root / 'docs' / 'EDA_REPORT.md', 'w') as f:
    f.write(summary.replace('║', '|').replace('╔', '').replace('╚', '').replace('╠', '').replace('╣', ''))

print(f"\n✓ EDA Report saved to: docs/EDA_REPORT.md")